In [3]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import sklearn
from scipy.optimize import curve_fit

import re

import ipywidgets as widgets

import os
import sys
sys.path.append('../liibattery3d')
import visualization

###### Load  and save Battery Performance excel sheets to pandas dataframes named "Battery_Performance_sheet_name"

In [4]:
Battery_Performance = pd.read_excel('NEW_3D_Battery_Performance_Log.xls', sheet_name = None, header=[0, 1, 2])

FileNotFoundError: [Errno 2] No such file or directory: 'NEW_3D_Battery_Performance_Log.xls'

In [5]:
Battery_Performance_sheet_names = pd.ExcelFile('NEW_3D_Battery_Performance_Log.xls').sheet_names

FileNotFoundError: [Errno 2] No such file or directory: 'NEW_3D_Battery_Performance_Log.xls'

In [ ]:
Battery_Performance_sheet_names

In [ ]:
for i, j in enumerate(Battery_Performance_sheet_names):
    globals()[f'Battery_Performance_{j}'] = Battery_Performance[Battery_Performance_sheet_names[i]]

###### Load and save Battery Performance excel sheets to pandas dataframes named "Battery_Parameter_sheet_name"

In [ ]:
Battery_Parameter = pd.read_excel('NEW_3D_Battery_Parameter_Log.xls', sheet_name = None)#, header=[0, 1, 2]) rework the excel file and come back to headers...

In [ ]:
Battery_Parameter_sheet_names = pd.ExcelFile('NEW_3D_Battery_Parameter_Log.xls').sheet_names

In [ ]:
Battery_Parameter_sheet_names

In [ ]:
for i, j in enumerate(Battery_Parameter_sheet_names):
    globals()[f'Battery_Parameter_{j}'] = Battery_Parameter[Battery_Parameter_sheet_names[i]]

###### Preview pandas dataframes

In [ ]:
## HINT: type "Battery_Performance_" or Battery_Parameter_", then use tab to quickly fill out the rest of a variable!

In [ ]:
#Battery_Performance_

In [ ]:
Battery_Parameter_MaterialParameters

In [ ]:
df = Battery_Performance_1_Discharge_Cover2

In [ ]:
len(df.columns)

In [ ]:
Battery_Parameter_GeometryParameters

#REFERENCES FOR MULTI-INDEXING

https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html

https://stackoverflow.com/questions/40554106/read-excel-sheet-with-multiple-header-using-pandas

In [ ]:
matplotlib.rcParams.update({'font.size': 10})
fig = plt.figure(figsize=(20,20))
#ax = fig.subplots(nrows=5,ncols=4)
#jet= plt.get_cmap('jet')
#colors = iter(jet(np.linspace(0,1,20)))

for i in range(int(len(df.columns)/2)):
    ax = plt.subplot(5, 4, i+1)
    ax.plot(df.iloc[:,2*i].values[1:],df.iloc[:,2*i+1].values[1:],color='k',marker='o', linewidth=0)
    #ax.set_xlim([0,40])
    #ax.set_ylim([0,200])
    #ax.set_xscale('log')
    #ax.set_yscale('log')

#### Define the Euqation to be fitted

In [ ]:
def objective(R,t,n,Q):
    return Q*(1-( (R*t) ** n ) * (1 - np.exp(- ((R*t) ** (-n)) )) )

In [ ]:
def plot_fit(R,y,t,n,Q):
    # plot data as scattered points
    plt.scatter(R, y)
    # define a sequence of x-axis point based on the limits of the data
    R_line = np.arange(0.5*min(R), 1.5*max(R), 0.01)
    # calculate y with the fitted equation
    y_line = objective(R_line, t, n, Q)
    # plot equation
    plt.plot(R_line, y_line, '--', color='red')
    return

In [ ]:
matplotlib.rcParams.update({'font.size': 18})
fig = plt.figure(figsize=(24,28))
#ax = fig.subplots(nrows=5,ncols=4)
list_of_t = []
list_of_n = []
list_of_Q = []
list_of_col = []
for i in range(int(len(df.columns)/2)):
    # choose the input and output variables
    R = df.iloc[:,2*i].values[0:]
    R = R[~pd.isnull(R)]
    y = df.iloc[:,2*i+1].values[0:]
    y = y[~pd.isnull(y)]
    if len(R)>=4:
        par, cov = curve_fit(objective, R, y, p0=[100,0.5,100])#, bounds=([-np.inf, 0, 0],[np.inf, 10, 500]))
        t,n,Q = par
        #print("\n\n")
        #par_std(t, n, Q, cov)
        std = np.sqrt(np.diag(cov))
        labs = '#'+str(i)+'\nt = ' + f'{t:.2f}' +'\nn = ' + f'{n:.2f}' + '\nQ = ' + f'{Q:.2f}'
        ax = plt.subplot(5, 4, i+1, title= '\nstd =' + f'{std[0]:.2f}'+ ','+ f'{std[1]:.2f}'+ ',' + f'{std[2]:.2f}')
        plt.text(0.1, 0.1, labs, transform=ax.transAxes)
        #ax.legend(lab,fontsize=18)
        ax = plot_fit(R, y, t, n, Q)
    else:
        t = 0
        n = 0
        Q = 0
        ax = plt.subplot(5, 4, i+1)
        plt.text(0.1, 0.1, 'not enough datapoints', transform=ax.transAxes)
        ax = plt.scatter(R,y)
    
    list_of_t.append(t)
    list_of_n.append(n)
    list_of_Q.append(Q)

### We can single out specific column that does not fit well with the general setting
We can set the initial point ***p0*** and parameter bounds ***bounds*** to cutomize the optimization alg.

In [ ]:
df.head(60)

In [ ]:
Battery_Parameter_GeometryParameters

In [ ]:
matplotlib.rcParams.update({'font.size': 18})
fig = plt.figure(figsize=(24,26))

# Specify case/dataset number and locate the R and y values
i=2
R = df.iloc[:,2*i].values[0:]
R = R[~pd.isnull(R)]
y = df.iloc[:,2*i+1].values[0:]
y = y[~pd.isnull(y)]

par, cov = curve_fit(objective, R, y, p0=[100,0.4,100])#, bounds=([-1, 0, 0],[np.inf, 10, 500]))
t,n,Q = par
std = np.sqrt(np.diag(cov))
labs = 't = ' + f'{t:.2f}' +'\nn = ' + f'{n:.2f}'+'\nQ = ' + f'{Q:.2f}'
ax = plt.subplot(5, 4, i+1)
plt.text(0.1, 0.1, labs, transform=ax.transAxes)
ax = plot_fit(R, y, t, n, Q)
print(std)

# Update the output fitting parameter
#list_of_t[9] = t
#list_of_n[9] = n
#list_of_Q[9] = Q


### Plot histograms to show distributions of n and tau
Figure 2(f) in Tian's paper

In [ ]:
matplotlib.rcParams.update({'font.size': 20})
fig = plt.figure(figsize=(16,6))
ax = fig.subplots(nrows=1,ncols=2)

ax[0].hist(list_of_n)
ax[0].set_xlabel('n')
ax[0].set_xlim(0,2.5)
ax[0].set_ylabel('number of cases')
ax[1].hist(list_of_t)
ax[1].set_xlabel('t')
ax[1].set_ylabel('number of cases')

### Place all Curve fitting parameters into a dataframe

In [ ]:
# Put all column names into a list
col_name = list(df.columns)[1::2]
# Turn the tuples into lists
for index,element in enumerate(col_name):
    col_name[index] = list(element[:-1])
    for sub_index,string in enumerate(col_name[index]):
        # conserve numbers only
        col_name[index][sub_index] = int(re.findall(r'\d+', string)[0])
    col_name[index] = tuple(col_name[index])
#col_name

### Place all fitting parameters into a dataframe following the format of the Parameter sheets

In [ ]:
par_df = pd.DataFrame(columns=['Paper #','Set','n','tau','Q'])
par_df
# Input paper and set numbers
for index,element in enumerate(col_name):
    row = [int(element[0]), int(element[1]), list_of_n[index], list_of_t[index], list_of_Q[index]]
    row_series = pd.Series(row, index=par_df.columns)
    par_df = par_df.append(row_series, ignore_index=True)
par_df

In [ ]:
visualization_df = pd.merge(par_df, Battery_Parameter_GeometryParameters, how="inner", on=['Paper #','Set'])
visualization_df = pd.merge(par_df, Battery_Parameter_Ge, how="inner", on=['Paper #','Set'])

visualization_df.head(40)

In [ ]:
# Replace NaN values
visualization_df = visualization_df.replace(['N/R'], np.NaN)

# This is the end of Curve fitting and the Beginning of Visualization

### Load Geometry and Material Parameters

In [ ]:
features = list(visualization_df.columns[6:])

In [ ]:
type(features)

In [ ]:
features

In [ ]:
####### Feature(s) to plot against n, tau, and Q

In [ ]:
visualization_features = widgets.SelectMultiple(
    options=features,
    #value = ,
    rows=10,
    description='Feature',
    disabled=False
)

In [ ]:
##### Hold 'SHIFT' or 'CTRL' to select multiple options then run cell

In [ ]:
visualization_features

In [ ]:
visualization_features = visualization_features.value

In [ ]:
visualization_features

In [ ]:
visualization.feature_vs_n_tau_Q(visualization_df, visualization_features)

In [ ]:
geo_df= pd.read_excel('NEW_3D_Battery_Parameter_Log.xls', sheet_name='GeometryParameters')

In [ ]:
geo_df.head(7)

In [ ]:
thickness_df = geo_df[['Paper #','Set','Cathode \nThickness [µm]','Anode \nThickness [µm]', 'Cathode 3D \nCharateristic Length [µm]','Anode 3D \nCharateristic\nLength [µm]']]
thickness_df.head(7)

### Include information on Architecture types

In [ ]:
# Select columns of interest
general_df = pd.read_excel('NEW_3D_Battery_Parameter_Log.xls', sheet_name='GeneralInformation')
arctype_df = general_df[['Paper #','Set','General Overall Architecture','General Cathode Architecture','General Anode Architecture','1C current Density [mA/cm^2]']]
arctype_df

In [ ]:
# Select columns of interest
general_df = pd.read_excel('NEW_3D_Battery_Parameter_Log.xls', sheet_name='GeneralInformation')
arctype_df = general_df[['Paper #','Set','General Overall Architecture','General Cathode Architecture','General Anode Architecture','1C current Density [mA/cm^2]']]
arctype_df

# Select rows of interest
arctype_df_needed = pd.DataFrame(columns=arctype_df.columns)
for index, row in par_df.iterrows():
    select_rows = arctype_df.loc[(arctype_df['Set']==row['Set']) & (arctype_df['Paper #']==row['Paper #'])]
    arctype_df_needed = arctype_df_needed.append(select_rows,ignore_index=True)

arctype_df_needed

par_df_full['General Overall Architecture'] = arctype_df_needed['General Overall Architecture']
par_df_full['General Cathode Architecture'] = arctype_df_needed['General Cathode Architecture']
par_df_full['General Anode Architecture'] = arctype_df_needed['General Anode Architecture']
par_df_full['1C current Density [mA/cm^2]'] = arctype_df_needed['1C current Density [mA/cm^2]']
par_df_full

### Include Material Information

In [ ]:
mat_df= pd.read_excel('NEW_3D_Battery_Parameter_Log.xls', sheet_name='MaterialParameters')
mat_df

In [ ]:
mat_df = mat_df[['Paper #','Set','Cathode Material','Anode Material']]
# Select rows of interest
mat_df_needed = pd.DataFrame(columns=mat_df.columns)
for index, row in par_df.iterrows():
    select_rows = mat_df.loc[(mat_df['Set']==row['Set']) & (mat_df['Paper #']==row['Paper #'])]
    mat_df_needed = mat_df_needed.append(select_rows,ignore_index=True)

mat_df_needed

par_df_full['Cathode Material'] = mat_df_needed['Cathode Material']
par_df_full['Anode Material'] = mat_df_needed['Anode Material']
par_df_full

In [ ]:
par_df_full[['Paper #','Set','Cathode Material','Anode Material']]

### Plot Fitting parameters versus cathode thickness
Figure 2(c) and Figure 2(e) in Tian's paper

In [ ]:
'''matplotlib.rcParams.update({'font.size': 16})
fig = plt.figure(figsize=(22,6))
ax = fig.subplots(nrows=1,ncols=3)

cathode_thickness = par_df_full['Cathode \nThickness [µm]'].values
anode_thickness = par_df_full['Anode \nThickness [µm]'].values

ax[0].scatter(cathode_thickness, n, color='g',s=200)
ax[0].set_xlabel('cathode thickness [µm]')
ax[0].set_ylabel('n')
for index, txt in enumerate(paper):
    ax[0].annotate(paper[index], (cathode_thickness[index], n[index]))
ax[0].semilogx()
    
ax[1].scatter(cathode_thickness, tau, color='g',s=200)
ax[1].set_xlabel('cathode thickness [µm]')
ax[1].set_ylabel('tau')
for index, txt in enumerate(paper):
    ax[1].annotate(paper[index], (cathode_thickness[index], tau[index]))
ax[1].loglog()
        
ax[2].scatter(cathode_thickness, Q, color='g',s=200)
ax[2].set_xlabel('cathode thickness [µm]')
ax[2].set_ylabel('Q')
for index, txt in enumerate(paper):
    ax[2].annotate(paper[index], (cathode_thickness[index], Q[index]))
ax[2].semilogx()'''

#### Plot without log scale

In [ ]:
matplotlib.rcParams.update({'font.size': 16})
fig = plt.figure(figsize=(22,6))
ax = fig.subplots(nrows=1,ncols=3)

cathode_arc = par_df_full['General Cathode Architecture'].values
anode_arc = par_df_full['General Anode Architecture'].values

ax[0].scatter(cathode_thickness, n, color='g',s=200)
ax[0].set_xlabel('cathode thickness [µm]')
ax[0].set_ylabel('n')
for index, txt in enumerate(paper):
    ax[0].annotate(paper[index], (cathode_thickness[index], n[index]))

    
ax[1].scatter(cathode_thickness, tau, color='g',s=200)
ax[1].set_xlabel('cathode thickness [µm]')
ax[1].set_ylabel('tau')
for index, txt in enumerate(paper):
    ax[1].annotate(paper[index], (cathode_thickness[index], tau[index]))

        
ax[2].scatter(cathode_thickness, Q, color='g',s=200)
ax[2].set_xlabel('cathode thickness [µm]')
ax[2].set_ylabel('Q')
for index, txt in enumerate(paper):
    ax[2].annotate(paper[index], (cathode_thickness[index], Q[index]))


### Plot fitting parameters versus 3D architecture characteristic length
Modifications of Figure 2(c) and Figure 2(e) in Tian's paper

In [ ]:
# Take either cathode or anode charateristic length
characteristic_length = []
char_cathode = par_df_full['Cathode 3D \nCharateristic Length [µm]'].values
char_anode = par_df_full['Anode 3D \nCharateristic\nLength [µm]'].values
for index,length in enumerate(char_cathode):
    if np.isnan(length)==False:
        characteristic_length.append(length)
    else: characteristic_length.append(char_anode[index])

matplotlib.rcParams.update({'font.size': 16})
fig = plt.figure(figsize=(22,6))
ax = fig.subplots(nrows=1,ncols=3)

ax[0].scatter(characteristic_length, n, color='g',s=200)
ax[0].set_xlabel('characteristic length [µm]')
ax[0].set_ylabel('n')
ax[0].semilogx()
for index, txt in enumerate(paper):
    ax[0].annotate(paper[index], (characteristic_length[index], n[index]))
    
ax[1].scatter(characteristic_length, tau, color='g',s=200)
ax[1].set_xlabel('characteristic length [µm]')
ax[1].set_ylabel('tau')
ax[1].loglog()
for index, txt in enumerate(paper):
    ax[1].annotate(paper[index], (characteristic_length[index], tau[index]))
    
ax[2].scatter(characteristic_length, Q, color='g',s=200)
ax[2].set_xlabel('characteristic length [µm]')
ax[2].set_ylabel('Q')
ax[2].semilogx()
for index, txt in enumerate(paper):
    ax[2].annotate(paper[index], (characteristic_length[index], Q[index]))        

#### Plot without log scale

In [ ]:
# Take either cathode or anode charateristic length
characteristic_length = []
char_cathode = par_df_full['Cathode 3D \nCharateristic Length [µm]'].values
char_anode = par_df_full['Anode 3D \nCharateristic\nLength [µm]'].values
for index,length in enumerate(char_cathode):
    if np.isnan(length)==False:
        characteristic_length.append(length)
    else: characteristic_length.append(char_anode[index])

matplotlib.rcParams.update({'font.size': 16})
fig = plt.figure(figsize=(22,6))
ax = fig.subplots(nrows=1,ncols=3)

ax[0].scatter(characteristic_length, n, color='g',s=200)
ax[0].set_xlabel('characteristic length [µm]')
ax[0].set_ylabel('n')

for index, txt in enumerate(paper):
    ax[0].annotate(paper[index], (characteristic_length[index], n[index]))
    
ax[1].scatter(characteristic_length, tau, color='g',s=200)
ax[1].set_xlabel('characteristic length [µm]')
ax[1].set_ylabel('tau')

for index, txt in enumerate(paper):
    ax[1].annotate(paper[index], (characteristic_length[index], tau[index]))
    
ax[2].scatter(characteristic_length, Q, color='g',s=200)
ax[2].set_xlabel('characteristic length [µm]')
ax[2].set_ylabel('Q')

for index, txt in enumerate(paper):
    ax[2].annotate(paper[index], (characteristic_length[index], Q[index]))        

### Figure 2(b) n vs tau, separated by material

In [ ]:
matplotlib.rcParams.update({'font.size': 16})
fig = plt.figure(figsize=(8,6))

plt.scatter(tau, n, color='g',s=200)
plt.xlabel('tau')
plt.ylabel('n')
#plt.semilogx()
texts = []
for index, txt in enumerate(paper):
    plt.annotate(paper[index], (tau[index], n[index]))     

# This code is used to adjust text annotations such that they don't overlap
'''
for index, txt in enumerate(cathode_arc):
    if txt != '1D':
        ann = 'c:'+str(txt)
        if str(anode_arc[index]) != '1D':
            ann = 'c:'+str(txt)+'\n'+'a:'+str(anode_arc[index])
    else: ann = 'a:'+str(anode_arc[index])
    texts.append(plt.text(tau[index], n[index], ann))
    #plt.annotate(ann, (tau[index], n[index]))
adjust_text(texts)
'''
